In [9]:
import pandas as pd 
import os
import sys
import pandas as pd 
import numpy as np
from torch.utils.data import *
from tqdm import tqdm

sys.path.append('../src/')
sys.path.append('..')

In [10]:
from src.models.lib.data import *
from src.helper import *

In [11]:
files = os.listdir('../data/processed/labels/')
files

['allen_cortex_labels.csv',
 'primary_bhaduri_labels.csv',
 'allen_m1_region_labels.csv',
 'whole_brain_bhaduri_labels.csv']

In [12]:
test2 = pd.read_csv('../data/processed/labels/allen_cortex_labels.csv').set_index('cell')
test2

,Type
cell,
1,7
2,7
3,7
4,7
5,7
...,...
49489,0
49490,7
49491,7


In [13]:
test2.iloc[47508, :]

Type    12
Name: 49493, dtype: int64

In [14]:
test = GeneExpressionData(
    filename='../data/interim/allen_cortex_T.csv',
    labelname='../data/processed/labels/allen_cortex_labels.csv',
    class_label='Type',
    cast=True,
    skip=3,
)

test.__getitem__(0)

(tensor([0., 0., 0.,  ..., 0., 0., 0.]), 7)

In [16]:
%%time

refgenes = gene_intersection()

CPU times: user 2.4 s, sys: 192 ms, total: 2.59 s
Wall time: 2.65 s


In [17]:
refgenes

['A1BG',
 'A1BG-AS1',
 'A2M',
 'A2M-AS1',
 'A4GALT',
 'AAAS',
 'AACS',
 'AADAT',
 'AAED1',
 'AAGAB',
 'AAK1',
 'AAMDC',
 'AAMP',
 'AANAT',
 'AAR2',
 'AARD',
 'AARS',
 'AARS2',
 'AARSD1',
 'AASDH',
 'AASDHPPT',
 'AASS',
 'AATF',
 'AATK',
 'ABAT',
 'ABCA1',
 'ABCA10',
 'ABCA13',
 'ABCA2',
 'ABCA3',
 'ABCA4',
 'ABCA5',
 'ABCA6',
 'ABCA7',
 'ABCA8',
 'ABCA9',
 'ABCB1',
 'ABCB10',
 'ABCB4',
 'ABCB5',
 'ABCB6',
 'ABCB7',
 'ABCB8',
 'ABCB9',
 'ABCC1',
 'ABCC10',
 'ABCC12',
 'ABCC2',
 'ABCC3',
 'ABCC4',
 'ABCC5',
 'ABCC6',
 'ABCC8',
 'ABCC9',
 'ABCD1',
 'ABCD2',
 'ABCD3',
 'ABCD4',
 'ABCE1',
 'ABCF1',
 'ABCF2',
 'ABCF3',
 'ABCG1',
 'ABCG2',
 'ABCG4',
 'ABHD1',
 'ABHD10',
 'ABHD11',
 'ABHD12',
 'ABHD12B',
 'ABHD13',
 'ABHD14A',
 'ABHD14B',
 'ABHD15',
 'ABHD16A',
 'ABHD17A',
 'ABHD17B',
 'ABHD17C',
 'ABHD2',
 'ABHD3',
 'ABHD4',
 'ABHD5',
 'ABHD6',
 'ABHD8',
 'ABI1',
 'ABI2',
 'ABI3',
 'ABI3BP',
 'ABL1',
 'ABL2',
 'ABLIM1',
 'ABLIM2',
 'ABLIM3',
 'ABO',
 'ABR',
 'ABRACL',
 'ABT1',
 'ABTB1',
 'ABT

In [22]:
len(set(refgenes).intersection(test.columns))

16604

In [23]:
from torch.utils.data import DataLoader, ConcatDataset

loader = DataLoader(test, batch_size=4)
sample = next(iter(loader))
sample = sample[0].numpy()

In [24]:
def clean_sample(sample, refgenes, currgenes):
    intersection = np.intersect1d(currgenes, refgenes, return_indices=True)
    indices = intersection[1] # List of indices in currgenes that equal refgenes 
    
    axis = (1 if sample.ndim == 2 else 0)
    sample = np.sort(sample, axis=axis)
    sample = np.take(sample, indices, axis=axis)

    return sample

In [25]:
datafiles, labelfiles = list(INTERIM_DATA_AND_LABEL_FILES_LIST.keys()), list(INTERIM_DATA_AND_LABEL_FILES_LIST.values())

datafiles = [os.path.join('..', 'data', 'interim', f) for f in datafiles]
labelfiles = [os.path.join('..', 'data', 'processed/labels', f) for f in labelfiles]
datafiles, labelfiles

(['../data/interim/primary_bhaduri_T.csv',
  '../data/interim/allen_cortex_T.csv',
  '../data/interim/allen_m1_region_T.csv',
  '../data/interim/whole_brain_bhaduri_T.csv'],
 ['../data/processed/labels/primary_bhaduri_labels.csv',
  '../data/processed/labels/allen_cortex_labels.csv',
  '../data/processed/labels/allen_m1_region_labels.csv',
  '../data/processed/labels/whole_brain_bhaduri_labels.csv'])

In [26]:
train = GeneExpressionData(datafiles[0], labelfiles[0], 'Type', skip=3)
loader = DataLoader(train, batch_size=4)
currgenes = train.columns

In [27]:
onedsample = train[0][0]
len(onedsample)

19765

In [28]:
%%timeit
(clean_sample(onedsample, refgenes, currgenes))

14 ms ± 159 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
twodsample = next(iter(loader))[0]
twodsample

tensor([[2.8467, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 1.8507, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [1.6067, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [30]:
%%timeit

clean_sample(twodsample, refgenes, currgenes)

14.9 ms ± 251 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
for X, y in tqdm(loader):
    X, y

  2%|███                                                                                                                                                | 985/46619 [00:19<15:20, 49.60it/s]


KeyboardInterrupt: 

In [32]:
for X, y in tqdm(loader):
    X = clean_sample(X, refgenes, currgenes)

  1%|█▍                                                                                                                                                 | 451/46619 [00:13<23:52, 32.23it/s]


KeyboardInterrupt: 

In [48]:
sample.ndim

2

In [84]:
temp = pd.read_csv(datafiles[0], nrows=1, header=1).columns 

In [104]:
cols = []
for file in datafiles:
    # Read in columns, split by | (since some are PVALB|PVALB), and make sure all are uppercase
    temp = pd.read_csv(file, nrows=1, header=1).columns 
    temp = [x.split('|')[0].upper().strip() for x in temp]
    
    print(f'Temp is {temp[0:5]}...')
    cols.append(set(temp))

unique = list(set.intersection(*cols))
unique = sorted(unique)

Temp is ['FO538757.2', 'AP006222.2', 'RP11-206L10.9', 'FAM41C', 'NOC2L']...
Temp is ['3.8-1.2', '3.8-1.3', '3.8-1.4', '3.8-1.5', '5-HT3C2']...
Temp is ['DDX11L1', 'WASH7P', 'MIR6859-1', 'MIR1302-2', 'FAM138A']...
Temp is ['RP11-34P13.7', 'FO538757.2', 'AP006222.2', 'RP4-669L17.10', 'RP11-206L10.9']...


In [105]:
len(unique)

16604

In [106]:
temp = pd.read_csv(datafiles[0], nrows=1, header=1).columns 
l = train.get_features()

In [107]:
all(temp == l)

True

In [108]:
len(set(unique).intersection(l))

16245

In [109]:
len(set(unique))

16604

In [111]:
len(set(unique).intersection([x.upper().strip() for x in l]))

16604